# テキストからSQLへのテスト

In [1]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT",""),
    api_key        = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version    = os.getenv("OPENAI_API_VERSION")
)

deployment_name    = os.getenv('DEPLOYMENT_NAME')

In [2]:
user_msg = """### Postgres SQLテーブルとそのプロパティ:
#
# Employee(id, name, department_id)
# Department(id, name, address)
# Salary_Payments(id, employee_id, amount, date)
#
### 過去3か月間に10人以上の従業員を雇用していた部門の名前をリストするクエリを作成します。
query:
"""

response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": user_msg},
    ],
  temperature=0,
  max_tokens=500
)

print(response.choices[0].message.content)

過去3か月間に10人以上の従業員を雇用していた部門の名前をリストするには、次のSQLクエリを使用できます:

```sql
SELECT d.name AS department_name
FROM Department d
JOIN Employee e ON d.id = e.department_id
WHERE e.id IN (
    SELECT DISTINCT employee_id
    FROM Salary_Payments
    WHERE date >= CURRENT_DATE - INTERVAL '3 months'
)
GROUP BY d.id, d.name
HAVING COUNT(e.id) > 10;
```

### 説明:
1. **`Salary_Payments` サブクエリ**:
   - 過去3か月間 (`date >= CURRENT_DATE - INTERVAL '3 months'`) に給与を受け取った従業員をフィルタリングします。
   - `DISTINCT` を使用して、ユニークな従業員のみを考慮します。

2. **`Employee` および `Department` テーブルの結合**:
   - 従業員を関連する部門に関連付けるために、`Employee` テーブルと `Department` テーブルを結合します。

3. **グループ化とカウント**:
   - 部門ごとに結果をグループ化します (`GROUP BY d.id, d.name`)。
   - `HAVING COUNT(e.id) > 10` を使用して、10人以上の従業員を持つ部門をフィルタリングします。

4. **部門名の選択**:
   - 条件を満たす部門名を選択します。


# SQLからテキストへのテスト

In [3]:
user_msg = """###このコードが何をしているのか説明できますか?
# コード: 
SELECT d.name
FROM Department d
WHERE d.id IN (
    SELECT e.department_id
    FROM Employee e
    WHERE e.id IN (
        SELECT sp.employee_id
        FROM Salary_Payments sp
        WHERE sp.date >= current_date - interval '3 months'
    )
    GROUP BY e.department_id
    HAVING COUNT(e.id) > 10
)
# 回答: 
"""
   
response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": user_msg},
    ],
   max_tokens=250)

print(response.choices[0].message.content)

このSQLコードは、`Department`、`Employee`、および `Salary_Payments` の3つのテーブルからデータを分析およびクエリしています。以下は、このコードの内容の詳細な説明です:

### 目標:
このクエリは、次の条件を満たす**部門** (`Department.d.name`) の`name`を取得します:
- **少なくとも10人の従業員**が、過去**3か月間**に**給与を受け取った**。
  
### クエリの内訳:
#### 1. **メインクエリ:**
```sql
SELECT d.name
FROM Department d
WHERE d.id IN (
    ...
)
```
- この部分は、`Department` テーブルから部門の名前 (`d.name`) を取得します。
- 条件として、部門の `id` が `WHERE` 節内の**サブクエリ**の結果と一致する必要があります。

---

#### 2. **最初のサブクエリ:**
```sql
SELECT e.department_id
FROM Employee e
WHERE e.id IN (
    ...
)
GROUP BY e.department_id
HAVING COUNT(e.id) > 10
```
- このサブクエリは、`Employee` テーブルを使用しています。
- 従業員を `department_id` ごとにグループ化し、その条件をチェックします。
